# FINANCIAL MARKET SIMULATION ON A SCALE-FREE TPI NETWORK

In this notebook we will explain the basics of our $TPI$ (Trade Position Influence) network model, based on a scale-free network.
We use it as our foundational financial market model. It will be tasked with performing the simulations (experiments).

We will explain the 3 main steps (functions) which need to be implemented for this model: (1) network initialization, (2) position updates, and (3) price updates.

### 1. NETWORK INITIALIZATION (function: initialize_network)

- a scale free network is generated using the Barabási-Albert model (algorithm) from Python's networkX package, ensuring the realistic trader connectivity base structure;

- nodes separated into 3 categories:

Hedge funds: 

- have a profit threshold determining when they switch positions
- trade in larger volumes (from HF_TRADE_LOWER to HF_TRADE_UPPER)

Traders:

- follow an influence-based position adjustment mechanism
- trade in smaller volumes (TR_TRADE_LOWER to TR_TRADE_UPPER)

Random traders:

- have no systematic strategy in this model
- randomly choose between buy and sell at each step

- each node has some initial position (either buy or sell)

- last_update_time attribute tracks when a trader last changed position
    


In [ ]:
def initialize_network():
    # 1. Construct scale-free network
    G = nx.barabasi_albert_graph(NUM_NODES, m=5)

    # 2. Assign node attributes
    num_random_traders = int(NUM_NODES * RANDOM_TRADER_RATIO)
    random_trader_indices = np.random.choice(
        range(NUM_HEDGE_FUNDS, NUM_NODES),
        size=num_random_traders, replace=False
    )
    for node in G.nodes:
        if node < NUM_HEDGE_FUNDS:
            G.nodes[node]['type'] = 'hedge_fund'
            G.nodes[node]['profit_threshold'] = np.random.normal(0.3, 0.1)
            G.nodes[node]['trade_size'] = np.random.uniform(HF_TRADE_LOWER, HF_TRADE_UPPER)
        elif node in random_trader_indices:
            G.nodes[node]['type'] = 'random_trader'
            G.nodes[node]['trade_size'] = np.random.uniform(TR_TRADE_LOWER, TR_TRADE_UPPER)
        else:
            G.nodes[node]['type'] = 'trader'
            G.nodes[node]['profit_threshold'] = np.random.normal(0.3, 0.1)
            G.nodes[node]['trade_size'] = np.random.uniform(0.2, 0.6)

        G.nodes[node]['last_update_time'] = 0
        G.nodes[node]['position'] = 'buy' if np.random.random() < 0.5 else 'sell'

    return G


### 2. POSITION UPDATES (update_positions)

Hedge funds:

- evaluate some profit at each step
- switch positions probabilistically based on a logistic-type function
- ensuring the hedge funds can adjust positions based on their profitability expectations

Traders:

- the influence on traders is modeled as:

$$
\text{Influence} = \alpha \times \frac{\text{neighbor's trade size}}{10} + \beta \times \text{number of neighbors}
$$
- traders update positions based on their neighbors' actions (dependent on trade size and network connectivity)
- random traders have a 50-50 probability of switching between buy and sell positions at every step; are not influenced by others, instead may represent other behavioral diversity which is not explained by our basic version of the TPI network model.



In [ ]:
def update_positions(t):
    for node in G.nodes:
        neighbors = list(G.neighbors(node))
        if G.nodes[node]['type'] == 'hedge_fund':
            profit = np.random.uniform(0,1)
            # Switch position if profit threshold is exceeded
            if np.random.rand() < 1 / (1 + np.exp(- (GAMMA * (profit - G.nodes[node]['profit_threshold'])))):
                G.nodes[node]['position'] = (
                    'buy' if G.nodes[node]['position'] == 'sell' else 'sell'
                )
                G.nodes[node]['last_update_time'] = t

        elif G.nodes[node]['type'] == 'trader':
            for neighbor in neighbors:
                influence = ALPHA * G.nodes[neighbor]['trade_size'] / 10 + BETA * len(neighbors)
                if np.random.rand() < influence and G.nodes[node]['last_update_time'] < t:
                    G.nodes[node]['position'] = G.nodes[neighbor]['position']
                    G.nodes[node]['last_update_time'] = t

        elif G.nodes[node]['type'] == 'random_trader':
            G.nodes[node]['position'] = 'buy' if np.random.rand() < 0.5 else 'sell'
            G.nodes[node]['last_update_time'] = t


### 3. MARKET PRICE UPDATE (update_price)

- the price movement is determined by the imbalance between buyers and sellers in the market;
- there is a volume-based adjustment mechanism implemented, where $P_{t+1}$ is the market price at step $t+1$:

$$
P_{t+1} = P_t + \eta \times (\text{buy volume} - \text{sell volume}) \times \text{exponential noise}
$$
- $\eta$ parameter controls price sensitivity
- exponential noise (scaled by the volume difference) ensures the random fluctuations while responding to supply and demand


In [ ]:
def update_price(prices, num_buyers, num_sellers):

        price = prices[-1]
        
        buyers = sum(1 for node in G.nodes if G.nodes[node]['position'] == 'buy')
        sellers = sum(1 for node in G.nodes if G.nodes[node]['position'] == 'sell')
   
        num_buyers.append(buyers)
        num_sellers.append(sellers)

        buy_volume = sum(G.nodes[node]['trade_size'] 
                         for node in G.nodes if G.nodes[node]['position'] == 'buy')
        sell_volume = sum(G.nodes[node]['trade_size'] 
                          for node in G.nodes if G.nodes[node]['position'] == 'sell')
        volume_difference = abs(buy_volume - sell_volume)

        # Avoid dividing by zero if volume_difference = 0
        if volume_difference == 0:
            volume_difference = 1

        price += ETA * (buy_volume - sell_volume) * np.random.exponential(1/volume_difference)
        prices.append(price)

        return prices, num_buyers, num_sellers

## Key takeaways

- TPI (Trade Position Influence) Model is formalized a network-based decision-making (buy/sell) model in which hedge funds use profit-based positioning of trades, while traders follow an influence-driven mechanism, and random traders input some randomness to the system, representing behavioral diversity otherwise not explained by the model.
- Our implementation captures well the market's microstructure, network-based trading strategies and key behavioral differences, as well as the adaptive price dynamics.
- The TPI network model can be used for analysis of emergent behavior in financial markets, continuing limited yet significant existing studies and imposes critical implications for future research, from fitting the model to real data to studying the impacts of regulations.

### Deriving our hypotheses

(1) AVALANCHE DETECTION AND ANALYSIS HYPOTHESIS

- this model generates price fluctuations driven by trade imbalances and the specific networked interactions on a scale-free network;
- wavelet transform analysis (found in wavelet.py) is used to detect abrubt changes in the price series by decomposing the signal into trend and residual components;
- the residual component captures sharp deviations, which correlate with high market activity, indicating avalanches;
- the avalanches are part of the market's macro-behavior, and are expected to follow a power-law  distribution, consistent with self-organized criticality 
- network-driven trading creates abrupt price shifts when influential nodes are active, and influence propagation causes coordinated position shifts leading to large cascades reflected in price movement

$H_1$ : Avalanches in price movement can be detected using wavelet transform analysis, where significant deviations in the residual signal correspond to periods of high market activity.


(2) FINANCIAL MARKETS AND SOC HYPOTHESIS

- in financial markets, studies have analyzed avalanches and found they often follow power-law distributions in size and duration;
- however in our analysis the inter-times also follow a power law (as in Bartolozzi et al. 2005), suggesting that instead of a Possion-like (memoryless) process, we observe temporal correlations, and what is described as a near-SOC or quasi-SOC state 

$H_2$ : The size and duration of market price avalanches exhibit scale-invariant behavior, following power-law distributions as observed in SOC systems. However, the market price avalanches of the simulated system do not emerge as true SOC, as indicated by the power-law distribution of avalanche inter-times


(3) TPI NETWORK AND STYLIZED FACTS HYPOTHESIS

- large price jumps (large returns) occur more frequently than in a normal distribution, which is a well known empirical fact in financial markets
- VOLATILITY CLUSTERING: the model exhibits periods of high and low volatility, where large price changes are followed by further price changes
- AUTOCORRELATION IN RETURS & SQUARED RETURNS: while raw returns show low autocorrelation, squared returns wxhibit stronger persistence as a key featrue of real financial markets

$H_3$ : A simple TPI network with heterogeneous traders (hedge funds, following traders, and random traders) is sufficient to reproduce key stylized facts of financial markets, including fat-tailed return distributions and volatility clustering, as well as autocorrelation in the returns and squared returns.